In [66]:
import os, numpy, csv, shutil
import pandas as pd
from statistics import mode

#Settings and directories
#Directories, !!! Make sure that you use "/" sclashes, not "\", Python does not like it. And finish all directories with /
#Example: 'C:/Users/31380/final/'
converter_Dir='C:/Users/Gedas/Desktop/DTU/31380/final/'
input_Dir='C:/Users/Gedas/Desktop/DTU/31380/final/input/'
output_Dir='C:/Users/Gedas/Desktop/DTU/31380/final/output/'

#Settings 'Relative' or 'Absolute'
time_Midi='Relative'


print('Converter:',converter_Dir+'midicsv.exe')
print('Input Midi files:'+input_Dir)
print('Output CSV files:'+output_Dir)

list_Midi=[0 for i in range(len(os.listdir(input_Dir)))]
list_Tempo=[0 for i in range(len(os.listdir(input_Dir)))]
i=0

for file in os.listdir(input_Dir):
    if file.endswith(".mid"):
        list_Midi[i]=file
        i=i+1
list_Midi=numpy.trim_zeros(list_Midi)
print(list_Midi)
if len(list_Midi)==0:
    print('Input directory is empty')

for the_file in os.listdir(output_Dir):
    file_path = os.path.join(output_Dir, the_file)
    try:
        if os.path.isfile(file_path) and the_file.endswith(".csv"):
            os.unlink(file_path)
    except Exception as e:
        print(e)
    
for i in range(0,len(list_Midi)):
    state=os.system(converter_Dir+'midicsv.exe '+input_Dir+list_Midi[i]+'>'+output_Dir+list_Midi[i].replace(".mid",".csv"))
    if state==1:
        print('Error: file '+list_Midi[i]+'convertion problem.')
        state=0
        
list_CSV=[0 for i in range(len(os.listdir(output_Dir)))]
i=0
for file in os.listdir(output_Dir):
    if file.endswith(".csv"):
        list_CSV[i]=file
        i=i+1
list_CSV=numpy.trim_zeros(list_CSV)

for i in range(0,len(list_CSV)):
    with open(output_Dir+list_CSV[i], 'r') as inp, open(output_Dir+list_CSV[i].replace(".csv","_e.csv"), 'w', newline='') as out:
        writer = csv.writer(out)
        for row in csv.reader(inp):
            if row[2] == " Note_on_c" or row[2] == " Note_off_c":
                writer.writerow(row)
            elif row[2] == " Tempo":
                list_Tempo[i]=row[3]
        
i=0                
for i in range(0,len(list_CSV)):
    data=pd.read_csv(output_Dir+list_CSV[i].replace(".csv","_e.csv"),
                     names=['Channel','Time','State','Something','Note','Volume','Duration','Bar_nr'],
                     index_col = False,  error_bad_lines=False, header=None, sep=',')
    data=data[['Channel','Time','State','Note','Volume','Duration','Bar_nr']]
    if time_Midi=='Relative':
        for n in range(0,len(data)):
            data.ix[n,'Bar_nr']=int((data.ix[n,'Time'])/(int(list_Tempo[i])/1000))
            try:
                note=data.ix[n,'Note']
                for g in range(1,10):
                    if data.ix[n+g,'Note'] == note and data.ix[n+g,'Volume'] == 0:
                        data.ix[n,'Duration']=data.ix[n+g,'Time']-data.ix[n,'Time']
                        break
                    elif g==10:
                        data.ix[n,'Duration']=1
            except Exception as e:
                e
        for n in range(0,len(data)):
            if data.ix[n,'State'] == ' Note_off_c' or (data.ix[n,'Volume'] == 0 and data.ix[n,'State'] == ' Note_on_c') or data.ix[n,'Duration'] == 0:
                data=data.drop(n)

    data.ix[-2,'State']='BPM'
    data.ix[-2,'Note']=int(60000/(int(list_Tempo[i])/1000))
    data.ix[-1,'State']='Tempo_ms'
    data.ix[-1,'Note']=(int(list_Tempo[i])/1000)
    data.to_csv(output_Dir+list_CSV[i].replace(".csv","_e.csv"), index = False)



Converter: C:/Users/Gedas/Desktop/DTU/31380/final/midicsv.exe
Input Midi files:C:/Users/Gedas/Desktop/DTU/31380/final/input/
Output CSV files:C:/Users/Gedas/Desktop/DTU/31380/final/output/
['deb_clai.mid', 'Hakuna.mid', 'midiout.mid']


In [15]:
from collections import Counter
data=pd.read_csv(output_Dir+'Hakuna_e.csv')
count=Counter(data['Time'])
print(count[mode(data['Time'])])